In [2]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [3]:
# Imports

import nflreadpy as nfl
import pandas as pd

## Retrieve data

In [4]:
schedule = nfl.load_schedules(seasons=2025)
schedule = schedule.to_pandas()

team_stats = nfl.load_team_stats(seasons=2025)
team_stats = team_stats.to_pandas()

In [5]:
team_stats

,season,week,team,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,...,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance
0,2025,1,ARI,REG,NO,21,29,163,2,0,...,2,2,0,0,1.0,0,0,0,0,0
1,2025,1,ATL,REG,TB,27,42,298,1,0,...,2,2,0,0,1.0,0,0,0,0,0
2,2025,1,BAL,REG,BUF,14,19,209,2,0,...,4,5,1,0,0.8,0,0,0,0,0
3,2025,1,BUF,REG,BAL,33,46,394,2,0,...,2,2,0,0,1.0,1,1,0,0,32
4,2025,1,CAR,REG,JAX,18,35,154,1,2,...,1,1,0,0,1.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2025,7,PHI,REG,MIN,19,23,326,3,0,...,4,4,0,0,1.0,0,0,0,0,0
208,2025,7,PIT,REG,CIN,23,34,249,4,2,...,4,4,0,0,1.0,0,0,0,0,0
209,2025,7,SF,REG,ATL,17,26,152,0,1,...,2,2,0,0,1.0,0,0,0,0,0
210,2025,7,TEN,REG,NE,25,34,255,1,1,...,1,1,0,0,1.0,0,0,0,0,0


In [6]:
# Filter out data I don't want

cols_to_drop = ['season', 
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [7]:
# for each week group by team sum/avg all previous weeks 
team_stats = team_stats.sort_values(["team", "week"]).reset_index(drop=True)

In [8]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att']

team_stats[[f"avg_{c}" for c in avg_cols]] = (
    team_stats.groupby("team")[avg_cols]
      .expanding()
      .mean()
      .reset_index(level=0, drop=True)
)


In [9]:
team_stats

,week,team,opponent_team,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,...,avg_def_tds,avg_def_fumbles,avg_def_safeties,avg_penalties,avg_penalty_yards,avg_punt_returns,avg_punt_return_yards,avg_kickoff_returns,avg_kickoff_return_yards,avg_fg_att
0,1,ARI,NO,21,29,163,2,0,5,-33,...,0.0,0.000000,0.000000,9.000000,54.000000,3.000000,30.000000,3.000000,73.000000,3.000000
1,2,ARI,CAR,17,25,220,1,1,1,-9,...,0.0,0.000000,0.000000,10.500000,75.000000,2.000000,29.500000,3.500000,80.000000,2.500000
2,3,ARI,SF,22,35,159,1,0,1,-5,...,0.0,0.000000,0.333333,8.666667,60.000000,1.666667,22.333333,3.666667,91.666667,2.666667
3,4,ARI,SEA,27,41,200,2,2,6,-36,...,0.0,0.000000,0.250000,8.250000,56.500000,1.500000,20.500000,4.000000,100.250000,2.500000
4,5,ARI,TEN,23,32,220,0,0,3,-28,...,0.0,0.200000,0.200000,8.200000,54.600000,1.600000,19.600000,4.000000,98.200000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,3,WAS,LV,15,21,207,1,0,1,-8,...,0.0,0.000000,0.000000,8.000000,57.666667,3.333333,59.000000,3.000000,102.333333,2.000000
208,4,WAS,ATL,16,27,156,2,1,2,-9,...,0.0,0.500000,0.000000,7.500000,55.750000,3.000000,44.250000,3.750000,126.500000,2.500000
209,5,WAS,LAC,15,26,231,1,0,1,-5,...,0.0,0.400000,0.000000,7.400000,54.200000,2.400000,35.400000,3.600000,119.600000,2.400000
210,6,WAS,CHI,19,26,211,3,1,3,-6,...,0.0,0.333333,0.000000,7.000000,51.833333,2.166667,31.000000,3.666667,120.833333,2.333333


## Add stats for previous weeks in schedule

In [11]:
# For week x in the schedule we need the offensive stats of 1 team and the defensive stats of the other team from week x - 1
